In [ ]:
import pandas as pd
from Preprocessing import *
from lightgbm import LGBMRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, ReLU
from keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, ReLU, BatchNormalization, LeakyReLU
import string
from utils import *

In [ ]:
train = pd.read_csv('data/train.csv')

In [ ]:
train, val = train_test_split(train, train_size=0.8, random_state=42)

train, val = One_hot_encoder(train, val, variable=['Education_Status', 'Employment_Status',
                                                    'Industry_Status', 'Occupation_Status', 'Race',
                                                    'Hispanic_Origin', 'Martial_Status', 'Household_Status',
                                                    'Household_Summary', 'Citizenship', 'Birth_Country',
                                                    'Birth_Country (Father)', 'Birth_Country (Mother)', 'Tax_Status', 'Income_Status'])

train['Gender'] = np.where(train['Gender'] == 'M', 1, 0)
val['Gender'] = np.where(val['Gender'] == 'M', 1, 0)

train = train.drop(['ID'], axis = 1)
val = val.drop(['ID'], axis = 1)

In [ ]:
variables = ['Education_Status', 'Employment_Status',
            'Industry_Status', 'Occupation_Status', 'Race',
            'Hispanic_Origin', 'Martial_Status', 'Household_Status',
            'Household_Summary', 'Citizenship', 'Birth_Country',
            'Birth_Country (Father)', 'Birth_Country (Mother)', 'Tax_Status', 'Income_Status'
            ]

In [ ]:
train.columns = train.columns.astype(str)
val.columns = val.columns.astype(str)

In [ ]:
X_train, y_train = train.drop('Income', axis = 1), train['Income']
X_val, y_val = val.drop('Income', axis = 1), val['Income']

In [ ]:
def Dimension_reduction(train, *args, n_reduced, first_node, second_node):    
    
    input_layer = Input(shape=(train.shape[1],))
    encoded = Dense(first_node)(input_layer)
    encoded = LeakyReLU()(encoded) 
    encoded = Dense(second_node)(encoded)
    encoded = LeakyReLU()(encoded) 
    
    encoded = Dense(n_reduced)(encoded)
    encoded = LeakyReLU()(encoded) 
    
    decoded = Dense(second_node)(encoded)
    decoded = LeakyReLU()(decoded) 
    decoded = Dense(first_node)(decoded)
    decoded = LeakyReLU()(decoded) 
    output_layer = Dense(train.shape[1], activation='elu')(decoded)

    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    autoencoder.fit(train, train, epochs=50, batch_size=16, verbose = False)

    encoder = Model(input_layer, encoded)
    
    column_names = list(string.ascii_lowercase)[:n_reduced]    
    train_reduced = pd.DataFrame(encoder.predict(train, verbose = False), columns=column_names)

    train = pd.concat([train, train_reduced], axis = 1)
    
    print('Reduced Col N :', len(train_reduced.columns))

    encoded_args = list()    
    if args:
        for data in args:

            data_reduced = pd.DataFrame(encoder.predict(data, verbose = False), columns=column_names)

            data = pd.concat([data, data_reduced], axis = 1)
        
            encoded_args.append(data)
    
    return [train, *encoded_args]

In [ ]:
X_train, X_val = Dimension_reduction(X_train, X_val, n_reduced=5, first_node=256, second_node=128, method='auto')

In [ ]:
model = LGBMRegressor(boosting_type = 'gbdt', learning_rate = 0.01, n_estimators=300)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(rmse)